In [26]:
from scipy.odr import Model, Data, ODR
import datetime as dt
from tree import TreeModel
from bau import DLWBusinessAsUsual
from cost import DLWCost
#from damage import DLWDamage
from utility import EZUtility
from optimization import GeneticAlgorithm, GradientSearch
import numpy as np

class matlabmode():
    def __init__(self,ind):
        '''init the class with default settings:
        1. decision time is set to [0, 15, 45, 85, 185, 285, 385]
        2. cost is using default x60 = 0.543, x100 = 0.671 and euro to dollar exchange rate = 1.2
        3. In the back stop tech model, join price is set to 2000, max prive is set to 2500, phi_0 = 1.5, phi_1 = 0 and constant = 30460
        3. constant growth of consumption is set to 0.015, subinterval length is 5 and ghg levels are 450,650,1000.
        4. Draws of simulation  = 4000000
        5. Disaster model's set up is  peak_temperature=6.0, disaster_tail=18.0. 
        6. Damage is simulated by pindcyk method with time to hit the max temperature = 100
        7. In utility, the parameter rho from the DLW-paper is set to 1-1/0.9, alpha is set to -6 and beta is set to 0.995^5 '''
        ind = int(ind)
        t = TreeModel(decision_times=[0, 15, 45, 85, 185, 285, 385])
        self.t = t 
        bau_default_model = DLWBusinessAsUsual()
        bau_default_model.bau_emissions_setup(tree=t)
        if ind == -1:
            from damage_fix_seed import DLWDamage
            c = DLWCost(t, bau_default_model.emit_level[0], g=92.08, a=3.413, join_price=2000.0, max_price=2500.0,
                            tech_const=1.5, tech_scale=0.0, cons_at_0=30460.0)
            df = DLWDamage(tree=t, bau=bau_default_model, cons_growth=0.015, ghg_levels=[450, 650, 1000], subinterval_len=5)    
            df.damage_simulation(draws=4000000, peak_temp=6.0, disaster_tail=18.0, tip_on=True, 
                                     temp_map=0, temp_dist_params=None, maxh=100.0)
            u = EZUtility(tree=t, damage=df, cost=c, period_len=5.0, eis=0.9, ra=7.0, time_pref=0.005)
            self.u = u
            paralist = np.array([[2.81, 4.6134, 6.14],[1.6667, 1.5974, 1.53139],[-0.25, -0.5, -1.0]])
        elif ind in [x for x in range(10)]:
            from damage_Yili import DLWDamage
            c = DLWCost(t, bau_default_model.emit_level[0], g=92.08, a=3.413, join_price=2000.0, max_price=2500.0,
                            tech_const=1.5, tech_scale=0.0, cons_at_0=30460.0)
            df = DLWDamage(tree=t, bau=bau_default_model, cons_growth=0.015, ghg_levels=[450, 650, 1000], subinterval_len=5,change=ind)    
            df.damage_simulation( draws=4000000, peak_temp=6.0, disaster_tail=18.0, tip_on=True, 
                                     temp_map=0, temp_dist_params=None, maxh=100.0)
            u = EZUtility(tree=t, damage=df, cost=c, period_len=5.0, eis=0.9, ra=7.0, time_pref=0.005)
            self.u = u
            paralist = np.array(self.u.damage.parameter_list)
        elif ind in (10,11):
            from damage import DLWDamage
            aa,bb,cost,g = self.sensitivity_analysis_c_k(ind)
            c = DLWCost(t, bau_default_model.emit_level[0], g=g, a=cost, join_price=2000.0, max_price=2500.0,
                    tech_const=1.5, tech_scale=0.0, cons_at_0=30460.0)
            df = DLWDamage(tree=t, bau=bau_default_model, cons_growth=0.015, ghg_levels=[450, 650, 1000], subinterval_len=5)    
            df.damage_simulation( draws=4000000, peak_temp=6.0, disaster_tail=18.0, tip_on=True, 
                                     temp_map=0, temp_dist_params=None, maxh=100.0)
            u = EZUtility(tree=t, damage=df, cost=c, period_len=5.0, eis=0.9, ra=7.0, time_pref=0.005)
            self.u = u
            paralist = np.array([aa,bb,cost,g])
        else:
            raise ValueError('Input indicator should be intergral within -1 to 11')
        self.parameters = paralist.ravel()
        #handle parameters:

    def sensitivity_analysis_c_k(self,ind):
        '''take fraction GHG reduction for different taxation rate from normal distribution
        returns the modified c and k in project description page 2 equation (2.3)'''
        #1.2 dollar = 1 euro
        xdata = [60*1.2,100*1.2]
        a = np.random.normal(0.543,0.0213)
        b = np.random.normal(0.671,0.0213)
        if ind == 0:
            ydata = [a,0.671]
        elif ind ==1:
            ydata = [0.543,b]
        else:
            ydata = [a,b]
        def f(p, x):
            '''Linear function y = m*x + b'''
            # B is a vector of the parameters.
            # x is an array of the current x values.
            # x is in the same format as the x passed to Data or RealData.
            #
            # Return an array in the same format as y passed to Data or RealData.
            return p[0] * x ** p[1]

        linear = Model(f)
        #sx, sy are arrays of error estimates
        mydata = Data(xdata, ydata)
        #beta0 are the initial parameter estimates
        myodr = ODR(mydata, linear, beta0=[1, -1.0])
        myoutput = myodr.run()
        x = myoutput.beta
        c= (1/x[1])*(x[1]+1) 
        g= ((1/(x[0]**(1/x[1])))**(x[1]+1) )*(x[0]-x[0]/(x[1]+1))
        return a,b,c,g

    def get_start_point(self):
        #use GA to get the start point for local optimizer
        ga_model = GeneticAlgorithm(pop_amount=150, num_generations=75, cx_prob=0.8, mut_prob=0.5, 
                              bound=1.5, num_feature=63, utility=self.u, print_progress=True) 
        final_pop, fitness = ga_model.run()
        sort_pop = final_pop[np.argsort(fitness)][::-1]
        begin_pop = final_pop[np.argsort(fitness)][-1]
        return sort_pop,begin_pop

    def utility_grad(self,m):
        #use finite differenciation to gradient and utility
        return self.u.utility(m),self.grad(m)

    def grad(self,m):
        #use finite differenciation to gradient and utility
        m = np.array(m)
        gs_model = GradientSearch(var_nums=63, utility=self.u, accuracy=1e-8, 
                              iterations=1, print_progress=True)
        grad = gs_model.numerical_gradient(m)
        return grad

    def utility(self,m):
        m = np.array(m)
        return self.u.utility(m)

    def GS(self,m):
        #m = np.array(m)
        m=m[0]
        gs_model = GradientSearch(var_nums=63, utility=self.u, accuracy=1e-8, 
                              iterations=200, print_progress=True)
        m_opt, u_opt = gs_model.run(initial_point_list=m, topk=1)
        return m_opt,u_opt

    def get_price(self,m):
        m = np.array(m)
        t = self.t
        price_list=list()
        for decision_time in range(len(t.decision_times)-1):
            start_node,end_node = t.get_nodes_in_period(decision_time)
            average_mit = self.u.damage.average_mitigation(m,decision_time)
            for index in range(end_node-start_node+1):
                index_ori =index + start_node
                price_list.append(self.u.cost.price(t.decision_times[decision_time],m[index_ori],average_mit[index]))
        return np.array(price_list)

    def utility_tree(self,m):
        # get utility in a tree structure from utlity class
        m = np.array(m)
        utility_tree = self.u.utility(m,True)[0]
        u_tree = utility_tree.tree
        utility_at_each_node = np.array([])
        for decision_time in self.t.decision_times[:-1]:
            utility_at_each_node = np.append(utility_at_each_node,u_tree[decision_time])

        return utility_at_each_node

    def utility_sub_optimal(self,m):
        # get utility from utlity class
        m = np.array(m)
        m = np.append(0,m)
        return self.u.utility(m), self.grad(m)

    def adj_utiltiy_cons(self,m,cons):
        # get utility from utlity class
        m = np.array(m)

        return self.u.adjusted_utility(m,first_period_consadj=cons)

In [27]:
y =matlabmode(9)

Starting damage simulation..
nothing is changed
Done!


In [33]:
get_utility_tree(np.ones((1,63))[0],y).

array([   9.11239894,   10.8225465 ,   11.73374651,   15.27802763,
         17.3087368 ,   17.3087368 ,   18.25065986,   23.28494447,
         28.1472818 ,   28.1472818 ,   30.57146494,   28.1472818 ,
         30.57146494,   30.57146494,   31.81512524,   61.549315  ,
         86.64732646,   86.64732646,  100.12692016,   86.64732646,
        100.12692016,  100.12692016,  107.55398822,   86.64732646,
        100.12692016,  100.12692016,  107.55398822,  100.12692016,
        107.55398822,  107.55398822,  111.12865047,  123.92117006,
        203.98124242,  203.98124242,  253.63080306,  203.98124242,
        253.63080306,  253.63080306,  281.45246073,  203.98124242,
        253.63080306,  253.63080306,  281.45246073,  253.63080306,
        281.45246073,  281.45246073,  297.07893709,  203.98124242,
        253.63080306,  253.63080306,  281.45246073,  253.63080306,
        281.45246073,  281.45246073,  297.07893709,  253.63080306,
        281.45246073,  281.45246073,  297.07893709,  281.45246

In [16]:
gs_model = GradientSearch(var_nums=63, utility=y.u, accuracy=1e-8, 
                      iterations=1, print_progress=True)
grad = gs_model.numerical_gradient(m)

In [22]:
y.utility_grad(m)


(array([ 9.3360125]),
 array([  2.34352981e-01,   4.22684110e-04,   6.57252031e-06,
          4.39381864e-04,   1.54898316e-04,   1.74527059e-04,
          9.68114477e-05,   4.29256630e-04,   1.42907908e-04,
          1.60227387e-04,   1.74527059e-04,   2.12274642e-04,
          6.21724894e-07,   4.51372273e-04,  -2.12807549e-04,
          1.25233157e-05,   2.38919995e-05,   9.59232693e-06,
          1.21325172e-04,  -9.59232693e-06,   8.66862138e-05,
         -2.69118061e-05,   4.42312853e-05,   3.28626015e-05,
         -2.51354493e-05,   7.22977234e-05,  -4.12114787e-05,
          2.71427325e-04,   1.99040784e-04,  -6.57252031e-06,
          1.97175609e-05,  -2.69118061e-05,   3.55271368e-06,
          1.07469589e-05,   3.55271368e-06,   9.50350909e-05,
         -2.57571742e-05,  -5.25801624e-05,   1.07558407e-04,
         -3.10862447e-05,  -2.69118061e-05,  -9.03277453e-05,
          4.79616347e-06,   3.23296945e-05,  -2.57571742e-05,
          2.21156427e-05,   1.55431223e-05,   2.

In [36]:
y.adj_utiltiy_cons(opt_m,-0.3641)

array([ 9.3378783])

In [33]:
y.utility(m)

array([ 9.33788787])

In [24]:
[0, 15, 45, 85, 185, 285, 385][:-1]

[0, 15, 45, 85, 185, 285]

In [25]:
np.array([])

array([], dtype=float64)

In [32]:
 m = np.array([0,
0.923249731147440,
0.754719483441788,
1.20883338344189,
1.04617671263710,
1.06283596501822,
0.764209719230075,
1.24802827326655,
1.21500326647379,
1.26575432525466,
1.22573195687188,
1.30171212654056,
1.25997799026830,
1.21904674024527,
0.403297122589292,
1.02577454258961,
0.994706872958886,
1.03145446568197,
0.994987147735444,
1.04674007627225,
1.01240549738639,
1.05967813466921,
1.02427403043297,
1.04805524739605,
1.01534566748373,
1.06280539211635,
1.02837250551094,
1.23652106519627,
1.20612936929712,
0.699894190865766,
0.436190781473611,
0.938789054590447,
0.924728408606642,
0.955867322283848,
0.930883850127116,
0.953693393191365,
0.929783215055321,
0.968473638588558,
0.949156242111391,
0.953178704410817,
0.929061076228053,
0.965782081245918,
0.944465733437724,
0.960192293601782,
0.938730199484934,
0.976157870014003,
0.897870204582257,
0.952111435670195,
0.928616743975446,
0.962061864574879,
0.941484004036435,
0.960785148412293,
0.936641727597721,
0.974519294962464,
0.871342018465326,
0.955319684920212,
0.933249837324057,
0.965177720803444,
0.937391371845865,
1.01241586127786,
0.305796220695870,
0.764616402965820,
0.181960386554750])

In [35]:
opt_m = [0.8338,
0.9868,
0.8448,
1.2945,
1.0234,
1.0398,
0.9092,
1.1073,
1.0714,
1.1788,
1.1350,
1.2050,
1.1633,
1.2114,
0.9166,
1.0259,
0.9896,
1.0291,
0.9897,
1.0347,
0.9973,
1.0505,
1.0142,
1.0397,
1.0032,
1.0530,
1.0171,
1.0581,
1.0236,
1.3148,
1.2111,
0.9423,
0.9250,
0.9618,
0.9366,
0.9600,
0.9348,
0.9787,
0.9559,
0.9559,
0.9311,
0.9708,
0.9538,
0.9647,
0.9452,
0.9838,
0.8843,
0.9552,
0.9302,
0.9693,
0.9507,
0.9624,
0.9447,
0.9815,
0.8856,
0.9623,
0.9412,
0.9776,
0.8342,
1.0005,
0.9768,
1.0889,
0.1412]

In [9]:
t = y.t
price_list=list()
for decision_time in range(len(t.decision_times)-1):
    start_node,end_node = t.get_nodes_in_period(decision_time)
    average_mit = y.u.damage.average_mitigation(m,decision_time)
    for index in range(end_node-start_node+1):
        index_ori =index + start_node
        price_list.append(y.u.cost.cost(decision_time,m[index_ori],average_mit[index]))

In [10]:
price_list

[0.0,
 0.095414989688338073,
 0.047958361241716728,
 0.15211982065144997,
 0.092892608545149996,
 0.098038895857966157,
 0.031803402367995282,
 0.092668537083086172,
 0.084563153292068163,
 0.097238201669419932,
 0.087138929404271259,
 0.10699361818793504,
 0.0957319986161803,
 0.085527509859731898,
 0.0019611774630505468,
 0.010467948777549368,
 0.0094248584754079868,
 0.010667102143889327,
 0.0094339251422015328,
 0.01121634605242096,
 0.010009589976305551,
 0.011696614267590744,
 0.010415779015804314,
 0.011264517460219867,
 0.010109151527580794,
 0.011814845397829442,
 0.010558711085450173,
 0.01980729559401612,
 0.018194442509022427,
 0.0028394677507586948,
 0.00056540186073647622,
 0.0017066288806075662,
 0.0016209548231863327,
 0.001814936800267434,
 0.0016580773025262406,
 0.0018008875380916911,
 0.0016513958745458181,
 0.0018979384397972599,
 0.0017718136583919875,
 0.0017975725955203668,
 0.0016470224717586919,
 0.0018799961657183172,
 0.0017421075671937068,
 0.00184311756694